In [ ]:
import pandas as pd
import numpy as np
import os

def create_target_dataset():
    """
    Move vd_zimc para última coluna e cria classificações WHO
    """
    
    # Configuração de caminhos
    input_path = '/Users/marcelosilva/Desktop/projectOne/4/C-Feature Removal/DatasetRemoval.csv'
    output_dir = '/Users/marcelosilva/Desktop/projectOne/4/D-Target Placing'
    output_path = os.path.join(output_dir, 'DatasetTarget.csv')
    
    # Criar diretório se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        # Carregar dataset
        df = pd.read_csv(input_path)
        print(f"Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas")
        
        # Verificar se vd_zimc existe
        if 'vd_zimc' not in df.columns:
            print("❌ ERRO: Coluna 'vd_zimc' não encontrada!")
            return None
        
        # Mover vd_zimc para última coluna
        vd_zimc_col = df['vd_zimc']
        df_reordered = df.drop('vd_zimc', axis=1)
        df_reordered['vd_zimc'] = vd_zimc_col
        
        print(f"✅ vd_zimc movido para última coluna")
        
        # Criar classificação WHO baseada no z-score IMC/idade
        def classify_nutritional_status_who(z_score):
            """
            Classifica estado nutricional baseado nos pontos de corte WHO
            """
            if pd.isna(z_score):
                return 'Missing'
            elif z_score < -2:
                return 'Desnutrido'  # Inclui magreza e magreza severa
            elif z_score >= -2 and z_score <= 1:
                return 'Peso adequado'
            elif z_score > 1 and z_score <= 2:
                return 'Sobrepeso'
            else:  # z_score > 2
                return 'Obesidade'
        
        # Aplicar classificação
        df_reordered['status_nutricional_who'] = df_reordered['vd_zimc'].apply(classify_nutritional_status_who)
        
        print(f"✅ Classificação WHO criada")
        
        # Relatório da classificação
        print(f"\n=== DISTRIBUIÇÃO DA CLASSIFICAÇÃO WHO ===")
        status_counts = df_reordered['status_nutricional_who'].value_counts()
        total_valid = len(df_reordered) - status_counts.get('Missing', 0)
        
        print(f"Total de registros: {len(df_reordered)}")
        print(f"Classificações válidas: {total_valid}")
        
        # Mostrar distribuição
        for status in ['Desnutrido', 'Peso adequado', 'Sobrepeso', 'Obesidade', 'Missing']:
            count = status_counts.get(status, 0)
            if total_valid > 0:
                percentage = (count / total_valid) * 100 if status != 'Missing' else (count / len(df_reordered)) * 100
            else:
                percentage = 0
            print(f"  {status}: {count} ({percentage:.1f}%)")
        
        # Estatísticas do z-score
        print(f"\n=== ESTATÍSTICAS DO Z-SCORE (vd_zimc) ===")
        z_score_stats = df_reordered['vd_zimc'].describe()
        print(f"Casos válidos: {df_reordered['vd_zimc'].notna().sum()}")
        print(f"Missing: {df_reordered['vd_zimc'].isna().sum()}")
        print(f"Média: {z_score_stats['mean']:.3f}")
        print(f"Desvio padrão: {z_score_stats['std']:.3f}")
        print(f"Mediana: {z_score_stats['50%']:.3f}")
        print(f"Min-Max: {z_score_stats['min']:.3f} - {z_score_stats['max']:.3f}")
        
        # Pontos de corte WHO aplicados
        print(f"\n=== PONTOS DE CORTE WHO APLICADOS ===")
        print(f"Desnutrido: z-score < -2.0")
        print(f"Peso adequado: z-score -2.0 a +1.0") 
        print(f"Sobrepeso: z-score +1.0 a +2.0")
        print(f"Obesidade: z-score > +2.0")
        
        # Verificar casos extremos
        extreme_cases = df_reordered[
            (df_reordered['vd_zimc'] < -3) | (df_reordered['vd_zimc'] > 3)
        ]
        if len(extreme_cases) > 0:
            print(f"\n⚠️  Casos extremos (|z-score| > 3): {len(extreme_cases)}")
            print(f"  Z-scores extremos: {extreme_cases['vd_zimc'].tolist()}")
        
        # Salvar dataset final
        df_reordered.to_csv(output_path, index=False)
        
        print(f"\n=== DATASET FINAL SALVO ===")
        print(f"Localização: {output_path}")
        print(f"Dimensões: {df_reordered.shape[0]} linhas × {df_reordered.shape[1]} colunas")
        
        # Verificar estrutura final
        print(f"\n=== ESTRUTURA FINAL DO DATASET ===")
        print(f"Features: {df_reordered.shape[1] - 2} colunas")
        print(f"Target z-score: vd_zimc (última coluna)")
        print(f"Target categórico: status_nutricional_who (nova coluna)")
        
        print(f"\n✅ Dataset Target criado com sucesso!")
        print(f"Pronto para análise e modelagem!")
        
        return df_reordered
        
    except Exception as e:
        print(f"❌ Erro ao processar dataset: {e}")
        return None

def main():
    """
    Função principal
    """
    print("=== CRIAÇÃO DO DATASET TARGET ===")
    print("Movendo vd_zimc para última coluna e criando classificações WHO\n")
    
    df_target = create_target_dataset()
    
    if df_target is not None:
        print(f"\n🎉 SUCESSO!")
        print(f"Dataset DatasetTarget.csv criado em:")
        print(f"/Users/marcelosilva/Desktop/projectOne/4/D-Target Placing/")
    else:
        print(f"\n❌ Erro na criação do dataset")

if __name__ == "__main__":
    main()